In [1]:
import pandas as pd
import pickle

import time

from pitched_recommend import Recommender

from pandas.io import gbq

In [2]:
with open('/Users/daria/Downloads/recommender%2Fmodel_20181112.pkl', 'rb') as f:
    model = pickle.load(f)

In [3]:
data_sql = "SELECT * FROM `umg-comm-tech-dev.adhoc.disney_country_rankings`"
data = gbq.read_gbq(data_sql, dialect='standard', project_id = 'umg-comm-tech-dev')

In [4]:
data_sql2 = "SELECT * FROM `umg-comm-tech-dev.adhoc.disney_country_rankings2`"
data2 = gbq.read_gbq(data_sql2, dialect='standard', project_id = 'umg-comm-tech-dev')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data2.head()

In [ ]:
data2.shape

In [5]:
data = pd.concat([data,data2], ignore_index=True)
data = data.drop_duplicates()
data.shape

(1349343, 8)

In [6]:
data['skips_perc'] = data.skips/data.streams
data['saves_perc'] = data.saves/data.streams

In [7]:
data = data[data.isrc.isin(model.isrcs)]
data = data.reset_index()

In [8]:
data.shape

(1079683, 11)

In [11]:
latam = ['Argentina','Peru','Venezuela','Chile','Ecuador','Guatemala','Cuba','Haiti','Bolivia','Dominican Republic',
         'Honduras','Paraguay','Nicaragua','El Salvador','Costa Rica','Panama','Puerto Rico','Uruguay']
aunz = ['Australia','New Zealand']

In [12]:
country_dict = {'spotify:user:128899670:playlist:3DcDiKi5jpRm958xYYUmVz':'Spain',
                'spotify:user:128899670:playlist:6hdO3Jed2HhOdDyZzWoVoz':'Mexico',
                'spotify:user:128899670:playlist:2qfycSO5TM8aA1FjZkDUSAp1':'Italy',
                'spotify:user:128899670:playlist:1WtZaYV0dPivolTfP4g5Tw':'Colombia',
                'spotify:user:128899670:playlist:7AEKbcsmnFH1a6LvlqUN8w': latam,
                'spotify:user:128899670:playlist:3EC7AzNCkupb9czLVQxuZa':'France',
                'spotify:user:128899670:playlist:1pqxs2DxVcLDdtWFQ4rp6N':'Brazil',
                'spotify:user:128899670:playlist:3rUyzoLplbar7sqYz6dIOZ':'Poland',
                'spotify:user:128899670:playlist:6BsFIlrKbCmJIVrJOARAz6':'Sweden',
                'spotify:user:128899670:playlist:3PrqjuND5YBvuq1U4gN6Io':'Belgium',
                'spotify:user:128899670:playlist:5JSHQs9z2jBfrwThqZe4Ft':'Norway',
                'spotify:user:128899670:playlist:5hEuQTqKTH78PODHovFDRb':'Netherlands',
                'spotify:user:128899670:playlist:5aYqqh0UAdYdYOtTUGSThv':'Denmark',
                'spotify:user:128899670:playlist:6Fd3hTl0Dy6kqVc3a1FC20':'Finland',
                'spotify:user:128899670:playlist:7njFsoqxecZmM6929TePcA': aunz,
                'spotify:user:128899670:playlist:7x3ki8zlAnNtNcXNPB6jns':'Philippines',
                'spotify:user:128899670:playlist:5ipH57xJ5XrWKgLoqUDC7q':'Indonesia',
                'spotify:user:128899670:playlist:7Hgc75ZscVfZRKHwizaScC':'Singapore',
                'spotify:user:128899670:playlist:4RLWGcrv63pBsCdnHgY2Sg':'Malaysia'}

In [13]:
scope = [
    'Spain',
    'Mexico',
    'Italy',
    'Colombia',
    'France',
    'Brazil',
    'Poland',
    'Sweden',
    'Belgium',
    'Norway',
    'Netherlands',
    'Denmark',
    'Finland',
    'Philippines',
    'Indonesia',
    'Singapore',
    'Malaysia',
    'LatAm',
    'AuNz'
]

In [17]:
sort_by_country = data.sort_values(by=['country_name','streams','skips_perc','saves_perc'], 
                                  ascending=[True,False,True,False])

In [18]:
top20_all = sort_by_country.groupby(by='country_name').head(20).reset_index()

In [20]:
top20_all['region'] = ''

for i in range(top20_all.shape[0]):
    if top20_all.country_name[i] in aunz:
        top20_all['region'][i]='AuNz'
    elif top20_all.country_name[i] in latam:
        top20_all['region'][i]='LatAm'
    elif top20_all.country_name[i] in scope:
        top20_all['region'][i]=top20_all['country_name'][i]
        
data.region.unique()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


array(['', 'AuNz', 'Philippines', 'Brazil', 'Sweden', 'Indonesia',
       'Mexico', 'Norway', 'Netherlands', 'Spain', 'Denmark', 'Singapore',
       'Italy', 'France', 'LatAm', 'Malaysia', 'Finland', 'Belgium',
       'Poland', 'Colombia'], dtype=object)

In [21]:
top_aunz = top20_all[top20_all.country_name.isin(aunz)].sort_values(by=['streams','skips_perc','saves_perc'],
                                                        ascending=[False,True,False])

top_latam = top20_all[top20_all.country_name.isin(latam)].sort_values(by=['streams','skips_perc','saves_perc'],
                                                        ascending=[False,True,False])

In [22]:
top30_aunz = top_aunz.groupby(by='isrc').max().reset_index().head(30)
top40_latam = top_latam.groupby(by='isrc').max().reset_index().head(40)

top20_all = top20_all[~top20_all.country_name.isin(aunz)]
top20_all = top20_all[~top20_all.country_name.isin(latam)]

top20_all = pd.concat([top20_all,top30_aunz,top40_latam], ignore_index=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [35]:
top20_all['days_on_playlist']=50
top20_all['playlist_uri']=top20_all['region']
top20_all = top20_all.rename(columns={'artist_name':'track_artist','track_name':'track_title'})

In [36]:
top20_scope = top20_all[top20_all.region!='']

In [37]:
top20_scope = top20_scope.drop(columns=['index','level_0'])

In [38]:
top20_scope.head()

,track_artist,country_name,isrc,region,saves,saves_perc,skips,skips_perc,streams,track_title,users,days_on_playlist,playlist_uri
360,Alessia Cara,Belgium,USWD11677601,Belgium,329954,0.051268,3512834,0.545821,6435870,How Far I'll Go - Alessia Cara Version,497496,50,Belgium
361,Elton John,Belgium,GBAMB7100001,Belgium,113693,0.032672,1314009,0.377607,3479834,Tiny Dancer,182491,50,Belgium
362,"Ariana Grande, John Legend",Belgium,USWD11778859,Belgium,156630,0.061907,1971054,0.779044,2530092,Beauty and the Beast,280554,50,Belgium
363,Idina Menzel,Belgium,USWD11366376,Belgium,124608,0.050231,1461262,0.589048,2480720,"Let It Go - From ""Frozen""/Soundtrack Version",183326,50,Belgium
364,Demi Lovato,Belgium,USWD11366344,Belgium,154224,0.090072,1330776,0.777217,1712232,"Let It Go - From ""Frozen / Single Version",430056,50,Belgium


In [39]:
rec_cols = ['isrc','track_artist','track_title','days_on_playlist','playlist_uri', 'country_name']
top20_all_recs = top20_scope[rec_cols]

In [40]:
top20_scope.to_csv('top20_by_country_final.csv')

In [41]:
gbq.to_gbq(dataframe=top20_scope, destination_table='disney.disney_top_by_country', 
           project_id='umg-comm-tech-dev', if_exists='replace')

1it [00:05,  5.23s/it]


In [42]:
rec=[]

for uri, country in country_dict.items():
    if type(country)==list:
        slice_recs = top20_all_recs[top20_all_recs.country_name.isin(country)].iloc[:,:-1].reset_index(drop='True')
        print("Producing recommendations for ", country)
        rec.extend(model.recommend_outofmodel(slice_recs, model.playlists_trainset, N=300))
    else:
        slice_recs = top20_all_recs[top20_all_recs.country_name==country].iloc[:,:-1].reset_index(drop='True')
        print("Producing recommendations for ", country)
        rec.extend(model.recommend_outofmodel(slice_recs, model.playlists_trainset, N=300))

Producing recommendations for  Spain


/usr/local/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


Producing recommendations for  Mexico
Producing recommendations for  Italy
Producing recommendations for  Colombia
Producing recommendations for  ['Argentina', 'Peru', 'Venezuela', 'Chile', 'Ecuador', 'Guatemala', 'Cuba', 'Haiti', 'Bolivia', 'Dominican Republic', 'Honduras', 'Paraguay', 'Nicaragua', 'El Salvador', 'Costa Rica', 'Panama', 'Puerto Rico', 'Uruguay']
Producing recommendations for  France
Producing recommendations for  Brazil
Producing recommendations for  Poland
Producing recommendations for  Sweden
Producing recommendations for  Belgium
Producing recommendations for  Norway
Producing recommendations for  Netherlands
Producing recommendations for  Denmark
Producing recommendations for  Finland
Producing recommendations for  ['Australia', 'New Zealand']
Producing recommendations for  Philippines
Producing recommendations for  Indonesia
Producing recommendations for  Singapore
Producing recommendations for  Malaysia


In [44]:
df_rec = model.rec_readable(rec)
df_rec.to_csv('disney_countries_recommendations.csv')

In [45]:
gbq.to_gbq(dataframe=df_rec, destination_table='disney.recommendations', 
           project_id='umg-comm-tech-dev', if_exists='replace')

1it [00:00,  2.54it/s]


In [46]:
df_rec.head()

,scores,isrc,artist,title,seed_playlist_uri
0,1.179070,USWD10110146,Phil Collins,You'll Be In My Heart,Spain
1,1.132929,USWD11677855,Auli'i Cravalho,How Far I'll Go,Spain
2,1.018199,USWD10935758,Miley Cyrus,The Climb,Spain
3,1.013767,USWD10210160,Joseph Williams,Can You Feel The Love Tonight,Spain
4,0.999624,USWD11677864,Auli'i Cravalho,Know Who You Are,Spain


In [47]:
df_rec.seed_playlist_uri.unique()

array(['Spain', 'Mexico', 'Italy', 'Colombia', 'LatAm', 'France',
       'Brazil', 'Poland', 'Sweden', 'Belgium', 'Norway', 'Netherlands',
       'Denmark', 'Finland', 'AuNz', 'Philippines', 'Indonesia',
       'Singapore', 'Malaysia'], dtype=object)

In [48]:
top20_scope.head()

,track_artist,country_name,isrc,region,saves,saves_perc,skips,skips_perc,streams,track_title,users,days_on_playlist,playlist_uri
360,Alessia Cara,Belgium,USWD11677601,Belgium,329954,0.051268,3512834,0.545821,6435870,How Far I'll Go - Alessia Cara Version,497496,50,Belgium
361,Elton John,Belgium,GBAMB7100001,Belgium,113693,0.032672,1314009,0.377607,3479834,Tiny Dancer,182491,50,Belgium
362,"Ariana Grande, John Legend",Belgium,USWD11778859,Belgium,156630,0.061907,1971054,0.779044,2530092,Beauty and the Beast,280554,50,Belgium
363,Idina Menzel,Belgium,USWD11366376,Belgium,124608,0.050231,1461262,0.589048,2480720,"Let It Go - From ""Frozen""/Soundtrack Version",183326,50,Belgium
364,Demi Lovato,Belgium,USWD11366344,Belgium,154224,0.090072,1330776,0.777217,1712232,"Let It Go - From ""Frozen / Single Version",430056,50,Belgium


In [49]:
top20_scope.playlist_uri.unique()

array(['Belgium', 'Brazil', 'Colombia', 'Denmark', 'Finland', 'France',
       'Indonesia', 'Italy', 'Malaysia', 'Mexico', 'Netherlands',
       'Norway', 'Philippines', 'Poland', 'Singapore', 'Spain', 'Sweden',
       'AuNz', 'LatAm'], dtype=object)

In [50]:
df = top20_scope.merge(df_rec, how='outer', left_on=['isrc','playlist_uri'], right_on = ['isrc','seed_playlist_uri'])

In [51]:
df.to_csv('rec_top_merge.csv')

In [52]:
rec_streams = pd.read_csv('/Users/daria/Downloads/adhoc_disney_streams.csv')
rec_streams.head()

,s_isrc,artist_name,track_name,country_name,streams,skips,saves,users
0,USMO19400306,The Jackson 5,I Want You Back,United States,1600128255,850788153,55417884,9541485
1,USIR19902220,Smash Mouth,All Star,United States,850885560,596371518,33249420,2001618
2,USMO17082628,The Jackson 5,ABC,United States,663454350,401734404,29336304,6766128
3,USMO19400306,The Jackson 5,I Want You Back,United Kingdom,649018737,375137418,20786415,7980327
4,USMO19400306,The Jackson 5,I Want You Back,UNKNOWN,570186708,333454680,0,7700526


In [53]:
rec_streams['skips_perc'] = rec_streams.skips/rec_streams.streams
rec_streams['saves_perc'] = rec_streams.saves/rec_streams.streams

In [54]:
rec_streams.head()

,s_isrc,artist_name,track_name,country_name,streams,skips,saves,users,skips_perc,saves_perc
0,USMO19400306,The Jackson 5,I Want You Back,United States,1600128255,850788153,55417884,9541485,0.531700,0.034633
1,USIR19902220,Smash Mouth,All Star,United States,850885560,596371518,33249420,2001618,0.700883,0.039076
2,USMO17082628,The Jackson 5,ABC,United States,663454350,401734404,29336304,6766128,0.605519,0.044218
3,USMO19400306,The Jackson 5,I Want You Back,United Kingdom,649018737,375137418,20786415,7980327,0.578007,0.032027
4,USMO19400306,The Jackson 5,I Want You Back,UNKNOWN,570186708,333454680,0,7700526,0.584817,0.000000


In [55]:
rec_labels = pd.read_csv('/Users/daria/Downloads/adhoc_disney_labels.csv')
rec_labels.head()

,isrc,major_label,label_studio,content_provider_name
0,QM6P41590607,Other,Ole Red Dot,BelieveSAS
1,USNLR1300792,Warner,WaterTower Music,WarnerBrosEnt
2,USWD11570628,Other,Now! Music,nowthatswhaticallmusicllp
3,USHR10421324,UMG,Hollywood Records Inc.,UMI
4,USMAC9300084,UMG,Polydor/Island,UMI


In [56]:
rec_data = rec_streams.merge(rec_labels, how='left', left_on = 's_isrc', right_on = 'isrc')
rec_data.head()

,s_isrc,artist_name,track_name,country_name,streams,skips,saves,users,skips_perc,saves_perc,isrc,major_label,label_studio,content_provider_name
0,USMO19400306,The Jackson 5,I Want You Back,United States,1600128255,850788153,55417884,9541485,0.531700,0.034633,USMO19400306,UMG,Universal Music Enterprises,Universal
1,USIR19902220,Smash Mouth,All Star,United States,850885560,596371518,33249420,2001618,0.700883,0.039076,USIR19902220,UMG,Universal Music Spain S.L.,UMI
2,USMO17082628,The Jackson 5,ABC,United States,663454350,401734404,29336304,6766128,0.605519,0.044218,USMO17082628,UMG,Motown,Universal
3,USMO19400306,The Jackson 5,I Want You Back,United Kingdom,649018737,375137418,20786415,7980327,0.578007,0.032027,USMO19400306,UMG,Universal Music Enterprises,Universal
4,USMO19400306,The Jackson 5,I Want You Back,UNKNOWN,570186708,333454680,0,7700526,0.584817,0.000000,USMO19400306,UMG,Universal Music Enterprises,Universal


In [57]:
rec_data.to_csv('/Users/daria/Downloads/disney_rec_data.csv', index=False)

In [59]:
len(rec_data.isrc.unique())

527

In [60]:
df_final = pd.read_csv('/Users/daria/Downloads/Disney Recommendations - top_20_and_recommendations.csv')
df_final.head()

,isrc,track_artist,track_title,country_name,region,saves,saves_perc,skips,skips_perc,streams,users,recommendation_scores,current_playlist_position
0,GBAMB7100001,Elton John,Tiny Dancer,New Zealand,AuNz,2927687.0,0.032938,41199260.0,0.463518,"88,883,818","590,728",top1,NaN
1,USWD11677601,Alessia Cara,How Far I'll Go - Alessia Cara Version,New Zealand,AuNz,1805570.0,0.034160,35672310.0,0.676025,"56,672,364","761,862",top2,NaN
2,USWD11366376,Idina Menzel,"Let It Go - From ""Frozen""/Soundtrack Version",New Zealand,AuNz,872498.0,0.030802,17298028.0,0.545769,"33,695,486","441,386",top3,8.0
3,USDW10500001,Lifehouse,You And Me,New Zealand,AuNz,685100.0,0.027314,12791880.0,0.495177,"26,919,440","215,760",top4,NaN
4,USWD11778859,"Ariana Grande, John Legend",Beauty and the Beast,New Zealand,AuNz,794190.0,0.033010,19673326.0,0.776729,"25,328,428","447,442",top5,11.0


In [64]:
df_final['saves'] = df_final['saves'].fillna(rec_data['saves'])
df_final['skips'] = df_final['skips'].fillna(rec_data['skips'])
df_final['saves_perc'] = df_final['saves_perc'].fillna(rec_data['saves_perc'])
df_final['skips_perc'] = df_final['skips_perc'].fillna(rec_data['skips_perc'])
df_final['streams'] = df_final['streams'].fillna(rec_data['streams'])
df_final['users'] = df_final['users'].fillna(rec_data['users'])

In [65]:
df_final = df_final.merge(rec_labels, how='left', on='isrc')

In [85]:
df_final.head()

,isrc,track_artist,track_title,country_name,region,saves,saves_perc,skips,skips_perc,streams,users,recommendation_scores,current_playlist_position,major_label,label_studio,content_provider_name
0,GBAMB7100001,Elton John,Tiny Dancer,New Zealand,AuNz,2927687.0,0.032938,41199260.0,0.463518,"88,883,818","590,728",top1,NaN,NaN,NaN,NaN
1,USWD11677601,Alessia Cara,How Far I'll Go - Alessia Cara Version,New Zealand,AuNz,1805570.0,0.034160,35672310.0,0.676025,"56,672,364","761,862",top2,NaN,NaN,NaN,NaN
2,USWD11366376,Idina Menzel,"Let It Go - From ""Frozen""/Soundtrack Version",New Zealand,AuNz,872498.0,0.030802,17298028.0,0.545769,"33,695,486","441,386",top3,8.0,NaN,NaN,NaN
3,USDW10500001,Lifehouse,You And Me,New Zealand,AuNz,685100.0,0.027314,12791880.0,0.495177,"26,919,440","215,760",top4,NaN,NaN,NaN,NaN
4,USWD11778859,"Ariana Grande, John Legend",Beauty and the Beast,New Zealand,AuNz,794190.0,0.033010,19673326.0,0.776729,"25,328,428","447,442",top5,11.0,NaN,NaN,NaN


In [69]:
df_final.label_studio.unique()

array([nan, 'Walt Disney Records', 'Universal Music Asia Pacific',
       'Hollywood Records', 'EMI Gold', 'EMI Marketing',
       'Universal Music S.A.', 'Now! Music', 'EMI Catalogue',
       'Villa 40/RCA Records', 'EMI TV', 'Lyric Street',
       'Republic/Universal Pictures/Sing', 'Dreamworks - Shrek II',
       'Universal Music Ltd.', 'Atlantic Records/ATG', 'We Love Disney',
       'Columbia', 'Hip-O (UC Admin)', 'Rhino', 'WM Singapore',
       'Polydor/Island', 'ARS Entertainment N.V.',
       'Universal Music Spain S.L.', 'Parlophone Catalogue',
       'UMOD (Universal Music On Demand)', 'Epic', 'EMI UK',
       'Fearless Records', 'Legacy Recordings', 'WM Australia',
       'Marketing Labs', 'Syco Music',
       'Universal Music Australia Pty. Ltd.', 'Sony Music Media',
       'RCA/Legacy', 'Sony Music UK', 'Hollywood Records Inc.',
       'Geffen Records', 'Polydor Records', 'Universal Music Enterprises',
       'Sony Music Catalog', 'XL', 'UMC (Universal Music Catalogue)',
 

In [75]:
labels_info = pd.read_csv('/Users/daria/Downloads/results-20190109-132202.csv')
labels_info.head()

,isrc,major_label,label_studio,content_provider_name
0,USWD10423200,UMG,EMI TV,umgglobal
1,USWD10527742,UMG,EMI TV,umgglobal
2,USWD10936987,UMG,EMI TV,umgglobal
3,USWD10527109,UMG,EMI TV,umgglobal
4,USWD10322246,UMG,EMI TV,umgglobal


In [77]:
current_labels = labels_info.label_studio.unique()

In [78]:
df_final.shape

(6090, 16)

In [86]:
df_final[df_final.label_studio.isin(current_labels)|df_final.label_studio.isnull()].shape

(4446, 16)

In [90]:
df_final_label_filters = df_final[df_final.label_studio.isin(current_labels)|df_final.label_studio.isnull()]

In [91]:
df_final_label_filters.groupby(by='region').recommendation_scores.nunique()

region
AuNz           223
Belgium        211
Brazil         241
Colombia       233
Denmark        225
Finland        202
France         260
Indonesia      211
Italy          271
LatAm          293
Malaysia       202
Mexico         256
Netherlands    220
Norway         235
Philippines    210
Poland         246
Singapore      221
Spain          261
Sweden         225
Name: recommendation_scores, dtype: int64

In [92]:
df_final_label_filters.to_csv('final_country_recommendations.csv')

In [93]:
df_final.head()

,isrc,track_artist,track_title,country_name,region,saves,saves_perc,skips,skips_perc,streams,users,recommendation_scores,current_playlist_position,major_label,label_studio,content_provider_name
0,GBAMB7100001,Elton John,Tiny Dancer,New Zealand,AuNz,2927687.0,0.032938,41199260.0,0.463518,"88,883,818","590,728",top1,NaN,NaN,NaN,NaN
1,USWD11677601,Alessia Cara,How Far I'll Go - Alessia Cara Version,New Zealand,AuNz,1805570.0,0.034160,35672310.0,0.676025,"56,672,364","761,862",top2,NaN,NaN,NaN,NaN
2,USWD11366376,Idina Menzel,"Let It Go - From ""Frozen""/Soundtrack Version",New Zealand,AuNz,872498.0,0.030802,17298028.0,0.545769,"33,695,486","441,386",top3,8.0,NaN,NaN,NaN
3,USDW10500001,Lifehouse,You And Me,New Zealand,AuNz,685100.0,0.027314,12791880.0,0.495177,"26,919,440","215,760",top4,NaN,NaN,NaN,NaN
4,USWD11778859,"Ariana Grande, John Legend",Beauty and the Beast,New Zealand,AuNz,794190.0,0.033010,19673326.0,0.776729,"25,328,428","447,442",top5,11.0,NaN,NaN,NaN
